<a href="https://colab.research.google.com/github/GiuliadeInnocentiis/Reinforcement-learning-and-conformal-prediction-Neural-Fitted-Q-Iteration-in-dynamic-systems/blob/main/Neural%20Fitted%20Q-Iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
np.random.seed(123)
torch.manual_seed(123)
python_random_seed = 123

# Load and prepare data
dati = pd.read_csv("water_tank.csv").iloc[:, 1:]
lambda_val = 0.2
dati['reward_new'] = dati['reward'] - lambda_val * (dati['stato_succ'] - dati['stato'])**2

# Split data
index = range(7200)
training = dati.iloc[index].copy()
test = dati.drop(index).copy()

# Parameters
gamma = 1  # discount factor
azioni_possibili = np.arange(0, 10, 0.5)
n_iter = 400 # number of FQI iterations

# Initialize metrics storage
mse = np.zeros(n_iter)
media_costo = np.zeros(n_iter)
std_costo = np.zeros(n_iter)
media_reward = np.zeros(n_iter)
std_reward = np.zeros(n_iter)

# Define groups for daily aggregation (24 hours)
gruppi = np.repeat(np.arange(len(training)//24), 24)

# Define neural network
class QNetwork(nn.Module):
    def __init__(self, input_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2= nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Prepare data
scaler = StandardScaler()
X_train = training[['stato', 'azione']].values
X_scaled = scaler.fit_transform(X_train)
y_train = training['reward_new'].values.reshape(-1, 1)

# Convert to PyTorch tensors
X_tensor = torch.FloatTensor(X_scaled)
y_tensor = torch.FloatTensor(y_train)
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Initialize model
input_size = X_train.shape[1]
q_mod = QNetwork(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(q_mod.parameters(), lr=0.001)

# Define price array
prices = np.array([1]*6 + [3]*2 + [2]*10 + [3]*2 + [2]*2 + [1]*2)
eta = 0.85

# Fitted Q-Iteration
# Get all unique states from the entire dataset
all_unique_states = pd.concat([training['stato'], training['stato_succ'], test['stato'], test['stato_succ']]).unique()

# Create a mapping from state to index
state_to_index = {state: i for i, state in enumerate(all_unique_states)}

for k in range(n_iter):
    print(f"Iteration FQI: {k+1}/{n_iter}")

    target_q = training['reward_new'].values.copy()

    if k > 0:
        # Pre-calcolo tutti i Q-values per stati unici
        # Process in batches for efficiency
        batch_size = 1024
        q_next = np.zeros((2, len(all_unique_states)))  # 2 rows: max_q and optimal_action

        for i in range(0, len(all_unique_states), batch_size):
            batch_states = all_unique_states[i:i+batch_size]

            # Create all possible state-action pairs
            state_action_pairs = np.array([
                [s, a] for s in batch_states for a in azioni_possibili
            ])

            # Scale and predict
            state_action_scaled = scaler.transform(state_action_pairs)
            with torch.no_grad():
                q_preds = q_mod(torch.FloatTensor(state_action_scaled)).numpy().flatten()
                q_preds = q_preds.reshape(len(batch_states), len(azioni_possibili))

                # Store max Q and optimal action
                q_next[0, i:i+len(batch_states)] = np.max(q_preds, axis=1)
                q_next[1, i:i+len(batch_states)] = azioni_possibili[np.argmax(q_preds, axis=1)]

        # Update target Q for non-terminal states
        not_done = training['done'] != 1
        # Use the state_to_index mapping to get the correct index for each successor state
        idx = [state_to_index[s] for s in training['stato_succ']]
        target_q[not_done] += gamma * q_next[0, idx][not_done]

        # Calculate daily costs and rewards
        temp_actions = q_next[1, idx]

        # Cost calculation
        costo_temp = -prices[np.tile(np.arange(24), len(training)//24)] * (1/eta) * (temp_actions**(1/3))
        costo_daily = np.array([np.sum(costo_temp[gruppi == g]) for g in np.unique(gruppi)])
        media_costo[k] = np.mean(costo_daily)
        std_costo[k] = np.std(costo_daily)

        # Reward calculation
        reward_temp = costo_temp - lambda_val * (training['stato_succ'] - training['stato'])**2
        reward_daily = np.array([np.sum(reward_temp[gruppi == g]) for g in np.unique(gruppi)])
        media_reward[k] = np.mean(reward_daily)
        std_reward[k] = np.std(reward_daily)


    # Update y_tensor with new target_q
    y_tensor = torch.FloatTensor(target_q.reshape(-1, 1))
    dataset = TensorDataset(X_tensor, y_tensor)

    # Train the model
    q_mod.train()
    for epoch in range(10):
        for X_batch, y_batch in dataloader:
            optimizer.zero_grad()
            outputs = q_mod(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

    # Compute MSE
    with torch.no_grad():
        predictions = q_mod(X_tensor)
        mse[k] = criterion(predictions, y_tensor).item()

# Optimal action function
def optimal_action(state, model, scaler):
    state_action_pairs = np.column_stack([
        np.full(len(azioni_possibili), state),
        azioni_possibili
    ])
    state_action_scaled = scaler.transform(state_action_pairs)
    with torch.no_grad():
        q_values = model(torch.FloatTensor(state_action_scaled)).numpy().flatten()
    return azioni_possibili[np.argmax(q_values)]

# Test the policy
test_states = test['stato'].values
test_actions = np.array([optimal_action(s, q_mod, scaler) for s in test_states])

# Calculate costs
test['hour'] = np.tile(np.arange(24), len(test)//24 + 1)[:len(test)] % 24
cost = prices[test['hour']] * (1/eta) * (test_actions**(1/3))

print("Cost difference:", np.sum(cost + test['reward'].values))

# Plot results
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(mse)
plt.title('MSE over Iterations')
plt.xlabel('Iteration')
plt.ylabel('MSE')

plt.subplot(1, 3, 2)
plt.plot(media_costo, label='Mean Cost')
plt.plot(media_costo + std_costo, 'r--', alpha=0.3)
plt.plot(media_costo - std_costo, 'r--', alpha=0.3)
plt.title('Daily Cost over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Cost')

plt.subplot(1, 3, 3)
plt.plot(media_reward, label='Mean Reward')
plt.plot(media_reward + std_reward, 'g--', alpha=0.3)
plt.plot(media_reward - std_reward, 'g--', alpha=0.3)
plt.title('Daily Reward over Iterations')
plt.xlabel('Iteration')
plt.ylabel('Reward')

plt.tight_layout()
plt.show()

In [ ]:
#Mi salvo i dati che servono per fare grafico reward
dati_reward = pd.DataFrame({'media_reward': media_reward, 'std_reward': std_reward, 'iterazione': np.arange(1, n_iter+1), 'lower':media_reward - std_reward, 'upper': media_reward + std_reward})
dati_reward.to_csv('dati_reward.csv', index=False)

In [ ]:
import numpy as np
import pandas as pd

# Optimal action function
def optimal_action(state, model, scaler):
    state_action_pairs = np.column_stack([
        np.full(len(azioni_possibili), state),
        azioni_possibili
    ])
    state_action_scaled = scaler.transform(state_action_pairs)
    with torch.no_grad():
        q_values = model(torch.FloatTensor(state_action_scaled)).numpy().flatten()
    return azioni_possibili[np.argmax(q_values)]


# Calcolo della domanda (equivalente a domanda <- test$stato + test$azione - test$stato_succ)
domanda = test['stato'] + test['azione'] - test['stato_succ']

# Inizializzazione degli array (equivalente a numeric(nrow(test)))
stato_nuovo = np.zeros(len(test)+1 )  # +1 perché salviamo anche lo stato finale
stato_nuovo[0] = test['stato'].iloc[0]  # Stato iniziale

azione_ottima = np.zeros(len(test))
done = test['done'].values
stato_oss = test['stato'].values

# Simulazione del sistema
for i in range(len(test)):
    azione_ottima[i] = optimal_action(stato_nuovo[i], q_mod, scaler)
    if done[i] == 0:
        # Caso non terminale
        stato_nuovo[i+1] = stato_nuovo[i] + azione_ottima[i] - domanda.iloc[i]
    else:

        if i + 1 < len(stato_oss):
            stato_nuovo[i+1] = stato_oss[i+1]
        else:
            # Se i+1 è fuori dai limiti, mantieni l'ultimo stato osservato
            stato_nuovo[i+1] = stato_oss[i]

# Analisi dei risultati (equivalente ai comandi R finali)
stato_nuovo = stato_nuovo[:-1]
# Tabella delle azioni ottimali
print("Distribuzione delle azioni ottimali:")
print(pd.Series(azione_ottima).value_counts().sort_index())

# Statistiche dello stato
print("\nStatistiche degli stati:")
print(pd.Series(stato_nuovo).describe())

# Conteggio stati <= 5 e >= 50
print(f"\nNumero di stati <= 5: {np.sum(stato_nuovo <= 5)}")
print(f"Numero di stati >= 50: {np.sum(stato_nuovo >= 50)}")
prices = np.concatenate([
    np.ones(6),       # rep(1,6)
    np.full(2, 3),    # rep(3,2)
    np.full(10, 2),   # rep(2,10)
    np.full(2, 3),    # rep(3,2)
    np.full(2, 2),    # rep(2,2)
    np.full(2, 1)     # rep(1,2)
])

eta = 0.85

# Calcolo del costo (adattato alla tua struttura)
# Assumendo che azione_ottima sia un array numpy di lunghezza 24*65=1560
costo = np.tile(prices, 65) * (1/eta) * np.power(azione_ottima, 1/3)

# Calcolo del costo totale (nota: - -test$reward equivale a + test$reward)
# Assumendo che test['reward'] sia una Series pandas o array numpy
total_cost = np.sum(costo + test['reward'].values)

print(f"Costo totale: {total_cost}")

In [ ]:
prices = np.concatenate([
    np.ones(6),       # rep(1,6)
    np.full(2, 3),    # rep(3,2)
    np.full(10, 2),   # rep(2,10)
    np.full(2, 3),    # rep(3,2)
    np.full(2, 2),    # rep(2,2)
    np.full(2, 1)     # rep(1,2)
])

eta = 0.85

# Calcolo del costo (adattato alla tua struttura)
# Assumendo che azione_ottima sia un array numpy di lunghezza 24*65=1560
costo = np.tile(prices, 65) * (1/eta) * np.power(azione_ottima, 1/3)

# Calcolo del costo totale (nota: - -test$reward equivale a + test$reward)
# Assumendo che test['reward'] sia una Series pandas o array numpy
total_cost = np.sum(costo + test['reward'].values)

print(f"Costo totale: {total_cost}")

Costo totale: -202.16463515831535


In [ ]:
#Faccio su tutto il dataset
import numpy as np
import pandas as pd

# Optimal action function
def optimal_action(state, model, scaler):
    state_action_pairs = np.column_stack([
        np.full(len(azioni_possibili), state),
        azioni_possibili
    ])
    state_action_scaled = scaler.transform(state_action_pairs)
    with torch.no_grad():
        q_values = model(torch.FloatTensor(state_action_scaled)).numpy().flatten()
    return azioni_possibili[np.argmax(q_values)]

# Calcolo della domanda (equivalente a domanda <- test$stato + test$azione - test$stato_succ)
domanda = dati['stato'] + dati['azione'] - dati['stato_succ']

# Inizializzazione degli array (equivalente a numeric(nrow(test)))
stato_nuovo = np.zeros(len(dati)+1 )  # +1 perché salviamo anche lo stato finale
stato_nuovo[0] = dati['stato'].iloc[0]  # Stato iniziale

azione_ottima = np.zeros(len(dati))
done = dati['done'].values

for i in range(len(dati)):
    azione_ottima[i] = optimal_action(stato_nuovo[i], q_mod, scaler)
    stato_nuovo[i+1] = stato_nuovo[i] + azione_ottima[i] - domanda.iloc[i]

# Analisi dei risultati (equivalente ai comandi R finali)
stato_nuovo = stato_nuovo[:-1]
# Tabella delle azioni ottimali
print("Distribuzione delle azioni ottimali:")
print(pd.Series(azione_ottima).value_counts().sort_index())
print(pd.Series(azione_ottima).describe())

# Statistiche dello stato
print("\nStatistiche degli stati:")
print(pd.Series(stato_nuovo).describe())

# Conteggio stati <= 5 e >= 50
print(f"\nNumero di stati <= 5: {np.sum(stato_nuovo <= 5)}")
print(f"Numero di stati >= 50: {np.sum(stato_nuovo >= 50)}")

In [ ]:
#Prezzo sul dataset completo
prices = np.concatenate([
    np.ones(6),       # rep(1,6)
    np.full(2, 3),    # rep(3,2)
    np.full(10, 2),   # rep(2,10)
    np.full(2, 3),    # rep(3,2)
    np.full(2, 2),    # rep(2,2)
    np.full(2, 1)     # rep(1,2)
])

eta = 0.85

# Calcolo del costo (adattato alla tua struttura)
# Assumendo che azione_ottima sia un array numpy di lunghezza 24*65=1560
costo = np.tile(prices, 365) * (1/eta) * np.power(azione_ottima, 1/3)

# Calcolo del costo totale (nota: - -test$reward equivale a + test$reward)
# Assumendo che test['reward'] sia una Series pandas o array numpy
total_cost = np.sum(costo + dati['reward'].values)

print(f"Costo totale: {total_cost}")

Costo totale: -285.96707664114524


In [ ]:
dati_azione=pd.DataFrame ({'azione' : azione_ottima, 'stato' : stato_nuovo, 'costo' : costo})
dati_azione.to_csv('dati_azione_sdg.csv', index=False)